In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()

In [ ]:
df = pl.DataFrame({'fm': ['a','b','c', None, None, None], 
                   'to': ['b','c','a', None, None, None],
                   'src':[None, None, None, 'x', 'y', 'z'],
                   'dst':[None, None, None, 'y', 'z', 'x']})
pos = {'a':(0,0), 'b':(1,0), 'c':(1,1), 'x':(10,10), 'y':(11,10), 'z':(11,11)}
_link_     = rt.link    (df, [('fm','to'),('src','dst')], pos)
_linknode_ = rt.linkNode(df, [('fm','to'),('src','dst')], pos)
rt.tile([_link_, _linknode_])

In [ ]:
df = pl.DataFrame({'fm': ['a','b','c'], 
                   'to': ['b','c','a']})
pos = {'a':(0,3), 'b':(5,3), 'c':(5,1), 'x':(10,10), 'y':(11,10), 'z':(11,11)}
_link2_     = rt.link    (df, [('fm','to')], pos)
_link2node_ = rt.linkNode(df, [('fm','to')], pos)
rt.tile([_link2_, _link2node_])

In [ ]:
df = pl.DataFrame({'fm': ['a','b','c'], 
                   'to': ['b','c','a']})
pos = {'a':(0,3), 'b':(5,3), 'c':(5,1)}
_link3_     = rt.link    (df, [('fm','to')], pos)
_link3node_ = rt.linkNode(df, [('fm','to')], pos)
rt.tile([_link3_, _link3node_])

In [ ]:
import random
import time

_fms_, _tos_ = [], []
for i in range(1_000_000):
    _fms_.append(random.randint(1,10_000))
    _tos_.append(random.randint(1,10_000))
df = pl.DataFrame({'fm':_fms_, 'to':_tos_})
pos = {}
for x in set(df['fm'])|set(df['to']): pos[x] = (random.random(), random.random())
t0 = time.time()
rt.link    (df, [('fm','to')], pos)._repr_svg_()
t1 = time.time()
rt.linkNode(df, [('fm','to')], pos)._repr_svg_()
t2 = time.time()

#
# link        |     linkNode      | date             | rows
# 2.7s        |     60.7s         | 2024-06-10 21:30 | 1M
#
print(f'link = {t1-t0} seconds | linkNode = {t2-t1} seconds')